In [1]:
from patcher.extractor import FateSeeker1Patcher, FateSeeker1PatchHelper, FateSeeker1MetaInfo, FateSeekerCsvParser
import UnityPy
import pathlib
import os
from collections import OrderedDict
import csv
import json
from pprint import pprint as pp

In [2]:
FateSeeker1MetaInfo.dlc_opening

'assets/forassetbundles/movies/other/dlcopening_chinese.bytes'

In [3]:
backuped_assets_path = pathlib.Path("extracted_assets")
fate_patcher = FateSeeker1Patcher(pathlib.Path(os.path.abspath(".")).joinpath("extracted_assets"))
fate_pach_helper = FateSeeker1PatchHelper(fate_patcher)
t = fate_patcher.get_text("assets/forassetbundles/textfiles/localization.csv")

In [4]:
def make_korean_string_table(input_file_path:str)->dict:
    output = dict()
    with open(input_file_path,"r",encoding="utf8") as f:
        for line in f:
            try:
                id,Key,Chinese,ENGLISH,Old,New, = line.strip().split("\t")
                output[Key] = New
            except Exception as e:
                print(line, e)
    return output

In [5]:
i = make_korean_string_table("./docs/input.tsv")

In [6]:
fs_localizations = FateSeekerCsvParser(t)

In [7]:
output = fs_localizations.change_by_key(i)

In [8]:
fate_patcher.set_text("assets/forassetbundles/textfiles/localization.csv", output)
fate_patcher.save_asset("./textfiles")

In [ ]:
output = list()
need_translations = dict()
with open("texts.csv",mode="r",encoding="utf8") as f:
    t = csv.DictReader(f, delimiter="\t")
    for row in t:
        if row["group"] in ("HEAR", "TNPC", "SDM", "SDRAMA", "fix"):
            if row["find"]:
                cf = row.get("find")
                print(row)
            # print([f"{row['group']}_{cf}"])
            need_translations.setdefault(f"{row['group']}_{cf}",OrderedDict())
            need_translations[f"{row['group']}_{cf}"][row["number"]] = row["Chinese"]




In [16]:
sliced_texts = OrderedDict()
for master_key, talks in need_translations.items():
    slice_number = 0
    total_length = 0
    for talk_id, script in talks.items():
        if total_length > 1400:
            slice_number += 1
            total_length = 0
        total_length += len(script)
        sliced_texts.setdefault(f"{master_key}#{slice_number}",OrderedDict())
        sliced_texts[f"{master_key}#{slice_number}"][talk_id] = script

In [ ]:
pp(sliced_texts)

json.dump(sliced_texts, open("sliced_texts.json", "w", encoding="utf8"), ensure_ascii=False, indent=4)

In [3]:
sliced_texts= json.load(open("sliced_texts.json","r",encoding="utf8"))

In [9]:
word_translation_dictionaries = dict()
with open("dictionary.csv", mode="r", encoding="utf8") as f:
    t = csv.DictReader(f, delimiter="\t")
    for row in t:
        word_translation_dictionaries[row["Chinese"]] = row["Translated"]
# pp(word_translation_dictionaries.values())

In [6]:
new_texts = dict()
meta_data_list = set()
for key, texts in sliced_texts.items():
    new_texts.setdefault(key, OrderedDict())
    new_texts[key]["need_translate"] = texts
    new_texts[key]["meta_data"] = set()
    for text_id, original_script in texts.items():
        for chinese_word in word_translation_dictionaries.keys():
            if chinese_word in original_script:
                new_texts.setdefault(key, OrderedDict())
                new_texts[key]["meta_data"].add((chinese_word, word_translation_dictionaries[chinese_word]))
        for v in ("姑娘", "小姐"):
            if v in original_script:
                meta_data_list.add(key)
    new_texts[key]["meta_data"] = {k:v for k,v in new_texts[key]["meta_data"]}


In [ ]:
pp(list(meta_data_list))

In [ ]:
for key in meta_data_list:
    os.remove(f"./works/{key}.txt")
    print(key)

In [35]:
# pp(new_texts)
json.dump(new_texts, open("script_with_meta_data.json", "w", encoding="utf8"), ensure_ascii=False, indent=4)

In [24]:
untranslated = dict()
with open("untranslated.csv", mode="r", encoding="utf8") as f:
    for row in f:
        id, text = row.split("\t")
        untranslated[id] = text

In [ ]:
untranslated

In [38]:
sliced_untranslated = OrderedDict()

total_length = 0
slice_number = 0
for id, script in untranslated.items():
    if total_length > 1000:
        slice_number += 1
        total_length = 0
    total_length += len(script)
    sliced_untranslated.setdefault(f"untranslated_{slice_number}",OrderedDict())
    sliced_untranslated[f"untranslated_{slice_number}"][id] = script

In [ ]:
sliced_untranslated

In [40]:
new_untranslated_texts = dict()
meta_data_list = set()
for key, texts in sliced_untranslated.items():
    new_untranslated_texts.setdefault(key, OrderedDict())
    new_untranslated_texts[key]["need_translate"] = texts
    new_untranslated_texts[key]["meta_data"] = set()
    for text_id, original_script in texts.items():
        for chinese_word in word_translation_dictionaries.keys():
            if chinese_word in original_script:
                new_untranslated_texts.setdefault(key, OrderedDict())
                new_untranslated_texts[key]["meta_data"].add((chinese_word, word_translation_dictionaries[chinese_word]))
    new_untranslated_texts[key]["meta_data"] = {k:v for k,v in new_untranslated_texts[key]["meta_data"]}

In [ ]:
new_untranslated_texts

In [42]:
json.dump(new_untranslated_texts, open("untranslated_with_meta_data.json", "w", encoding="utf8"), ensure_ascii=False, indent=2)

In [ ]:
pp(list(new_untranslated_texts.keys()))